In [1]:
import pandas as pd

df = pd.read_csv('goodreads_train.csv')
df.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1


In [2]:
df.columns

Index(['user_id', 'book_id', 'review_id', 'rating', 'review_text',
       'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes',
       'n_comments'],
      dtype='object')

In [3]:
df['rating'].value_counts()

4    313688
5    265007
3    188972
2     72627
0     30988
1     28718
Name: rating, dtype: int64

In [4]:
def train_test_split(df):
    valcounts = df['rating'].value_counts()
    df_test = df.copy()
    for i in range(6):
        indx_tot = df[df.rating.eq(i)].index
        indx_dif = []
        indx = df[df.rating.eq(i)].sample(valcounts[i]-20102).index
        df.drop(indx, inplace=True)
        for val in indx_tot:
            if val not in indx:
                indx_dif.append(val)
        df_test.drop(indx_dif, inplace=True)
    print('Train set values:')
    print(df['rating'].value_counts())
    print('Test set values:')
    print(df_test['rating'].value_counts())
    return(df, df_test)

df_train, df_test = train_test_split(df)
del df

Train set values:
0    20102
5    20102
4    20102
3    20102
2    20102
1    20102
Name: rating, dtype: int64
Test set values:
4    293586
5    244905
3    168870
2     52525
0     10886
1      8616
Name: rating, dtype: int64


Veo primero si leyeron el libro

In [5]:
print(f'No empezaron y no terminaron {len(df_train.query("read_at.isnull() and started_at.isnull()", engine="python"))/len(df_train)*100 :.2f}%')

No empezaron y no terminaron 15.86%


In [6]:
print(f'Terminaron pero no empezaron {len(df_train.query("read_at.isnull() and not started_at.isnull()", engine="python"))/len(df_train)*100 :.2f}%')

Terminaron pero no empezaron 3.16%


In [7]:
print(f'Empezaron y no terminaron {len(df_train.query("not read_at.isnull() and started_at.isnull()", engine="python"))/len(df_train)*100 :.2f}%')

Empezaron y no terminaron 20.49%


In [8]:
print(f'Empezaron y terminaron {len(df_train.query("not read_at.isnull() and not started_at.isnull()", engine="python"))/len(df_train)*100 :.2f}%')

Empezaron y terminaron 60.49%


In [9]:
import re
import string   # libreria de cadena de caracteres
import contractions

def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = contractions.fix(text)
    
    # pasa las mayusculas del texto a minusculas
    text = text.lower()                                              
    # reemplaza texto entre corchetes por espacio en blanco.. ¿ y \% no se..
    text = re.sub('\[.*?¿\]\%', ' ', text)                           
    # reemplaza signos de puntuacion por espacio en blanco.. %s -> \S+ es cualquier caracter que no sea un espacio en blanco
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text) 
    # remueve palabras que contienen numeros.
    text = re.sub('\w*\d\w*', '', text)
    
    # Sacamos comillas, los puntos suspensivos, <<, >>
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    return text

round0 = lambda x: clean_text(x)

# Dataframe que resulta de aplicarle a las columnas la funcion de limpieza
review_text = pd.DataFrame(df_train.review_text.apply(round0))

df_train.drop('review_text', axis=1, inplace=True)

df_train['review_text'] = review_text

del(review_text)

In [10]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer() # funcion para lematizar
nltk.download('stopwords') # hay que descargar este modulo en particular
sw = nltk.corpus.stopwords.words('english') # lista de stopwords

def remove_sw(text):
    words = text.split(' ') # separamos por espacios
    words_clean = []
    for w in words:
        if not w in sw: # si no es stopword, agregamos la version lematizada
            words_clean.append(lemmatizer.lemmatize(w))
    return ' '.join(words_clean)

round3 = lambda x: remove_sw(x)
 
review_text =  pd.DataFrame(df_train.review_text.apply(round3))

df_train.drop('review_text', axis=1, inplace=True)

df_train['review_text'] = review_text

del(review_text)

[nltk_data] Downloading package wordnet to /home/kenneths/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kenneths/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
import time
from gensim.models import Word2Vec

size = 300
stemmed_tokens = pd.Series(df_train['review_text']).values
# Train the Word2Vec Model
w2v_model = Word2Vec(min_count=1, # ignora palabras cuya frecuencia es menor a esta
                     vector_size=size,
                     sample=6e-5, # umbral para downsamplear palabras muy frecuentes
                     alpha=0.03, # tasa de aprendizaje inicial (entrenamiento de la red neuronal)
                     min_alpha=0.0007, # tasa de aprendizaje minima
                     negative=20) # penalidad de palabras muy frecuentes o poco informaitvas

In [12]:
w2v_model.build_vocab(df_train.review_text)

In [13]:
w2v_model.train(df_train.review_text,
                epochs=30,
                total_examples=w2v_model.corpus_count)

(82963732, 2232517110)

In [14]:
def buildWordVector(text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += w2v_model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [15]:
from sklearn.preprocessing import scale
import numpy as np
n_dim = 300
train_vecs = np.concatenate([buildWordVector(z, n_dim) for z in df_train.review_text])
train_vecs = scale(train_vecs)

In [16]:
test_vecs = np.concatenate([buildWordVector(z, n_dim) for z in df_test.review_text])
test_vecs = scale(test_vecs)

In [17]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(train_vecs, df_train.rating)

RandomForestClassifier()

In [18]:
from sklearn.metrics import classification_report

y_prob =  clf.predict(test_vecs)

print(classification_report(df_test.rating,y_prob))

              precision    recall  f1-score   support

           0       0.03      0.50      0.06     10886
           1       0.01      0.26      0.03      8616
           2       0.08      0.20      0.12     52525
           3       0.26      0.17      0.20    168870
           4       0.42      0.15      0.22    293586
           5       0.41      0.22      0.28    244905

    accuracy                           0.18    779388
   macro avg       0.20      0.25      0.15    779388
weighted avg       0.35      0.18      0.22    779388

